# Problem: Lemonade Change

Statement
There is a lemonade stand where customers can buy one lemonade at a time for 
$
5
$5
 and pay with a 
$
5
$5
, 
$
10
$10
, or 
$
20
$20
 bill. It is necessary to return the correct change to each customer so that the net transaction is completed successfully with a total payment of 
$
5
$5
. Note that no change is available initially.

Given an integer array, bills, where bills[i] represents the bill paid by the 
i
t
h
i 
th
 
 customer, return TRUE if it is possible to provide every customer with the correct change, or FALSE otherwise.

Constraints:

1
≤
1≤
 bills.length 
≤
500
≤500

bills[i] is either 
5
5
, 
10
10
, or 
20
20
.

Hint:
- You might want to go over the Greedy Techniques pattern again.

In [ ]:
from collections import defaultdict

def lemonade_change(bills):
    available_bills = defaultdict(int)
    
    for bill in bills:
        if bill == 5:
            available_bills[5] += 1
        
        elif bill == 10:
            if available_bills[5] == 0:
                return False
            available_bills[5] -= 1
            available_bills[10] += 1
        
        elif bill == 20:
            if available_bills[10] > 0 and available_bills[5] > 0:
                available_bills[10] -= 1
                available_bills[5] -= 1
            elif available_bills[5] >= 3:
                available_bills[5] -= 3
            else:
                return False
    
    return True


In [5]:
bills = [5,5,5,5,20,10,10]
lemonade_change(bills)

False

In [ ]:
# Better optimized solution
def lemonade_change(bills):
    five, ten = 0, 0   # counters for $5 and $10 bills we have
    
    for bill in bills:
        if bill == 5:
            five += 1
        elif bill == 10:
            if five == 0:
                return False
            five -= 1
            ten += 1
        else:  # bill == 20
            if ten > 0 and five > 0:
                ten -= 1
                five -= 1
            elif five >= 3:
                five -= 3
            else:
                return False
    
    return True


# Problem: Finding MK Average

You are given two integers, m and k, and a stream of integers. Your task is to design and implement a data structure that efficiently calculates the MK Average for the stream.

To compute the MK Average, follow these steps:

Stream length check: If the stream contains fewer than m elements, return -1 as the MK Average.

Window selection: Otherwise, copy the last m elements of the stream to a separate container and remove the smallest k elements and the largest k elements from the container.

Average calculation: Calculate the average of the remaining elements (rounded down to the nearest integer).

Implement the MKAverage class

MKAverage(int m, int k): Initializes the object with integers m and k and an empty stream.

void addElement(int num): Adds the integer num to the stream.

int calculateMKAverage(): Returns the current MK Average for the stream as described above, or -1 if the stream contains fewer than m elements.

Constraints:

3
<
=
3<=
 m 
<
=
1
0
5
<=10 
5
 

1
<
1<
 k*2 
<
m
<m

1
<
=
1<=
 num 
<
=
1
0
5
<=10 
5
 

1
0
3
10 
3
 
 calls will be made to addElement and calculateMKAverage, at most.

In [ ]:
from collections import deque
import bisect
class MKAverage(object):
    def __init__(self, m, k):
        self.m = m
        self.k = k
        self.container = deque()
        self.sortedList = []
        self.midSum = 0


    def addElement(self, num):
        self.container.append(num)

        bisect.insort(self.sortedList, num)

        if len(self.container)>self.m: #remove oldest from stream container and sortedList
            old = self.container.popleft()
            idx = bisect.bisect_left(self.sortedList,old)
            self.sortedList.pop(idx)
        
        if len(self.container) == self.m:
            self._compute_mid_sum()
        
    
    def _compute_mid_sum(self):
        self.midSum = sum(self.sortedList[self.k:self.m-self.k])

    
    def calculateMKAverage(self):
        if len(self.container)<self.m:
            return -1
        return self.midSum // (self.m - 2* self.k)

Explanation

- We maintain a deque (stream) to track last m elements in order.
- We maintain a sorted_list (acts as a BST):
  - Insert new number with bisect.insort (O(m)).
  - Remove oldest number using bisect.bisect_left (O(m)).
To compute MKAverage:
- Take slice sorted_list[k : m-k].
- Compute sum → average.

# Leetcode's problem 187. Repeated DNA Sequences

The DNA sequence is composed of a series of nucleotides abbreviated as 'A', 'C', 'G', and 'T'.

For example, "ACGAATTCCG" is a DNA sequence.
When studying DNA, it is useful to identify repeated sequences within the DNA.

Given a string s that represents a DNA sequence, return all the 10-letter-long sequences (substrings) that occur more than once in a DNA molecule. You may return the answer in any order.

 

Example 1:

Input: s = "AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT"
Output: ["AAAAACCCCC","CCCCCAAAAA"]
Example 2:

Input: s = "AAAAAAAAAAAAA"
Output: ["AAAAAAAAAA"]
 

Constraints:

1 <= s.length <= 105
s[i] is either 'A', 'C', 'G', or 'T'.


In [ ]:
# Ali's solution: using hash tables
from collections import defaultdict
def findRepeatedDnaSequences(s):
    if len(s)<10:
        return []
    l,h = 0, 9
    n = len(s)
    seen = set()
    res = set()
    while h<n:
        
        curr = s[l:h+1]
        if curr in seen:
            res.add(curr)
        else:
            seen.add(curr)

        l+=1
        h+=1
    
    return list(res)

In [26]:
s = "AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT"
findRepeatedDnaSequences(s)

['AAAAACCCCC', 'CCCCCAAAAA']

In [27]:
s = "AAAAAAAAAAAAA"
findRepeatedDnaSequences(s)

['AAAAAAAAAA']

Great — let’s add the **intuition**, the **step-by-step approach**, and inline **comments** so it’s crystal clear why this works and why it’s fast.

---

## ✨ Intuition

* The DNA alphabet has only 4 letters (`A, C, G, T`).
* Each letter can be represented in **2 bits**:

  * `A → 00`, `C → 01`, `G → 10`, `T → 11`.
* So any 10-letter DNA substring can be represented in **20 bits** (fits easily in a normal integer).
* Instead of slicing strings of length 10 at every step (which is costly), we encode them into integers and slide a rolling hash window.
* We use a bitmask to **drop old characters** when sliding and keep only the last 20 bits.
* A set tracks hashes we’ve seen; if a hash appears again, the substring is repeated.

---

## 🪜 Step-by-Step Approach

1. **Edge case:** If string is shorter than 10, no repeats possible.
2. **Initialize mapping:** `A, C, G, T` → 0,1,2,3.
3. **Build first window (10 chars):** Left-shift and append bits for each character.
4. **Store first hash in `seen`.**
5. **Slide through the rest of the string:**

   * Shift left by 2 bits, append new char bits.
   * Apply a mask `(1 << 20) - 1` to keep only 20 bits (last 10 chars).
   * If hash is already in `seen`, substring is a repeat → add it to `res`.
   * Otherwise add hash to `seen`.
6. **Return all repeats as a list.**

---

## 📘 Code with Comments

```python
class Solution:
    def findRepeatedDnaSequences(self, s: str) -> List[str]:
        # No possible repeats if shorter than 10
        if len(s) < 10:
            return []

        # Map each nucleotide to 2 bits
        bitMap = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

        # Mask to keep only 20 bits (10 chars * 2 bits)
        mask = (1 << 20) - 1  
        hash_val = 0  # rolling hash value

        seen = set()  # hashes seen once
        res = set()   # substrings seen more than once

        # Encode the first 10 characters into the initial hash
        for i in range(10):
            hash_val = (hash_val << 2) | bitMap[s[i]]
        seen.add(hash_val)

        # Slide the window over the rest of the string
        for i in range(10, len(s)):
            # Shift left by 2, add new char bits, and mask off old chars
            hash_val = ((hash_val << 2) | bitMap[s[i]]) & mask

            if hash_val in seen:
                # Found a repeat: extract substring (only when needed)
                res.add(s[i-9:i+1])
            else:
                # First time seeing this 10-char sequence
                seen.add(hash_val)

        # Convert set to list for output
        return list(res)
```

---

## 📊 Complexity

* **Time:** O(n) → one pass over the string.
* **Space:** O(n) → storing up to O(n) hashes and repeated substrings.
* **Optimization gain:** avoids creating n substrings of length 10; works mostly with integers.

---

👉 Do you want me to also show the **bit-decoding alternative** (instead of slicing `s[i-9:i+1]`) with comments, so you have both approaches fully documented?


In [ ]:
# Ali's solution using bit encoding trick (A=0, C=1, G=2, T=3) and bitmask/rolling hash:

def findRepeatedDnaSequences(s):
    if len(s)<10:
        return []
    bitMap = {'A':0, 'C':1, 'G':2, 'T':3}
    mask = (1 << 20) - 1   # 20 ones in binary
    hash_val = 0

    seen = set()
    res = set()

    for i in range(10):
        hash_val = (hash_val << 2) | bitMap[s[i]]
    seen.add(hash_val)
    for i in range(10,len(s)):
        hash_val = ((hash_val << 2) | bitMap[s[i]]) & mask
        if hash_val in seen:
            res.add(s[i-9:i+1])
        else:
            seen.add(hash_val)
    return list(res)


In [33]:
s = "AAAAACCCCCAAAAACCCCCCAAAAAGGGTTT"
findRepeatedDnaSequences(s)

['AAAAACCCCC', 'CCCCCAAAAA']

# Leetcode's Problem 416. Partition Equal Subset Sum

Given an integer array nums, return true if you can partition the array into two subsets such that the sum of the elements in both subsets is equal or false otherwise.

 

Example 1:

Input: nums = [1,5,11,5]
Output: true
Explanation: The array can be partitioned as [1, 5, 5] and [11].
Example 2:

Input: nums = [1,2,3,5]
Output: false
Explanation: The array cannot be partitioned into equal sum subsets.
 

Constraints:

1 <= nums.length <= 200
1 <= nums[i] <= 100

In [ ]:
class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        